In [ ]:
pip install keras==2.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 336 kB 28.6 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires keras<2.9,>=2.8.0rc0, but you have keras 2.2.5 which is incompatible.


In [ ]:
pip install tensorflow==1.14.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 50 kB/s 
     |████████████████████████████████| 3.1 MB 54.8 MB/s 
     |████████████████████████████████| 488 kB 67.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel('Final_DL_Data_1.xlsx')

In [ ]:
data['metaphor_label'] = data['metaphor_label'].fillna(0)

In [ ]:
data.isna().sum()

Unnamed: 0               0
Unnamed: 0.1             0
Unnamed: 0.1.1           0
final_tweets             1
label                    0
metaphor_label           0
simile_label             0
clause_polarity_label    0
punctuation_count        0
positive_words           0
negative_words           0
opposite_polarity        0
sequence                 0
is_intensifier           0
interjection_count       0
mixed_case               0
repeated_letters         0
repeated_words           0
dtype: int64

In [ ]:
data.dropna(inplace=True)

**TABNET**

In [ ]:
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import os
import torch
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
X = data.drop(['Unnamed: 0','Unnamed: 0.1.1','Unnamed: 0.1','final_tweets','label'],axis=1)
y = data['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=14)

In [ ]:
X= X.to_numpy()
y= y.to_numpy() 
y= y.flatten()

In [ ]:
kf = KFold(n_splits=5, random_state=14, shuffle=True)
CV_score_array  =[]
for train_index, test_index in kf.split(X):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    tb_cls = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=0.0025),
                       scheduler_params={"step_size":10, "gamma":0.4},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type="entmax"
                       )
    tb_cls.fit(X_train,y_train,
               eval_set=[(X_train, y_train), (X_valid, y_valid)],
               eval_name=['train', 'valid'],
               eval_metric=['accuracy'],
               max_epochs=100 , patience=0,
               batch_size=25, drop_last=False)            

Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.8476  | train_accuracy: 0.52185 | valid_accuracy: 0.53287 |  0:00:03s
epoch 1  | loss: 0.66011 | train_accuracy: 0.60407 | valid_accuracy: 0.6263  |  0:00:06s
epoch 2  | loss: 0.60411 | train_accuracy: 0.71051 | valid_accuracy: 0.72145 |  0:00:09s
epoch 3  | loss: 0.54262 | train_accuracy: 0.74513 | valid_accuracy: 0.75606 |  0:00:12s
epoch 4  | loss: 0.51851 | train_accuracy: 0.75552 | valid_accuracy: 0.76125 |  0:00:15s
epoch 5  | loss: 0.50337 | train_accuracy: 0.76158 | valid_accuracy: 0.77336 |  0:00:18s
epoch 6  | loss: 0.49676 | train_accuracy: 0.76677 | valid_accuracy: 0.76298 |  0:00:21s
epoch 7  | loss: 0.4967  | train_accuracy: 0.76417 | valid_accuracy: 0.75606 |  0:00:24s
epoch 8  | loss: 0.48825 | train_accuracy: 0.76417 | valid_accuracy: 0.76125 |  0:00:27s
epoch 9  | loss: 0.47759 | train_accuracy: 0.77109 | valid_accuracy: 0.77855 |  0:00:30s
epoch 10 | loss: 0

In [ ]:
np.mean(CV_score_array)

In [ ]:
predictions =[ 0 if i < 0.5 else 1 for i in tb_cls.predict(X_test.to_numpy())]

In [ ]:
(y_test==predictions).mean() #test accuracy

In [ ]:
X_val_probas=tb_cls.predict_proba(X_valid) #probability of validation classes

In [ ]:
X_train_probas=tb_cls.predict_proba(X_train) #probability of training classes

In [ ]:
X_test_probas=tb_cls.predict_proba(X_test.to_numpy()) #probability of test classes

In [ ]:
X_val_probas.shape

In [ ]:
X_valid.shape

In [ ]:
type(y_train)

numpy.ndarray

DEEP-STACK

In [ ]:
pip install deepstack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import deepstack
from deepstack.base import KerasMember
from deepstack.base import Member
from tensorflow.keras.utils import to_categorical
from deepstack.base import KerasMember
from deepstack.ensemble import StackEnsemble
from deepstack.ensemble import DirichletEnsemble

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score

dirichletEnsemble

In [ ]:
dirichletEnsemble = DirichletEnsemble(N=2000*4,metric=accuracy_score)

In [ ]:
model1 = tb_cls# A Keras pre-trained Model (Base-Learner)
# A numpy tuple (X_val, y_val) or Keras Data Iterator - Validation Data for Meta-Learner
member1 = Member(name="TABNET", train_probs=X_train_probas,train_classes=y_train,val_classes=y_valid,val_probs=X_val_probas,submission_probs=X_test_probas)

In [ ]:
model2 = m #(Base-Learner)
member2 = Member(name="1-D CNN", train_probs=predict_prob_train1,train_classes=Y1_train,val_classes=Y1_val,val_probs=predict_prob_val1,submission_probs=predict_prob_test1)

In [ ]:
model3 = mlp #(Base-Learner)
member3 = Member(name="MLP", train_probs=X2_train_prob,train_classes=y2_train,val_classes=y2_valid,val_probs=X2_valid_prob,submission_probs=X2_test_prob)

In [ ]:
dirichletEnsemble.add_member(member1)
dirichletEnsemble.add_member(member2)
dirichletEnsemble.add_member(member3)

In [ ]:
dirichletEnsemble.fit()

In [ ]:
dirichletEnsemble.describe()

TABNET - Weight: 0.6383 - accuracy_score: 0.8580
1-D CNN - Weight: 0.0006 - accuracy_score: 0.8007
MLP - Weight: 0.3610 - accuracy_score: 0.8603
DirichletEnsemble accuracy_score: 0.8779


IndexError: ignored

Stacked Ensemble

1D CNN

In [ ]:
 X1= data.drop(['Unnamed: 0','Unnamed: 0.1.1','Unnamed: 0.1','final_tweets','label'],axis=1)
 Y1 = data['label']

In [ ]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, test_size=0.33, random_state=14)

In [ ]:
X1_train, X1_val, Y1_train, Y1_val = train_test_split(X1_train, Y1_train, test_size=0.29855, random_state=1) # 0.25 x 0.67= 0.16

In [ ]:
X1_train = np.array(X1_train).reshape(-1,13,1)
X1_test = np.array(X1_test).reshape(-1,13,1)

In [ ]:
print(X1_train.shape,X1_test.shape)

(1358, 13, 1) (954, 13, 1)


In [ ]:
X1_val.to_numpy().shape

(578, 13)

In [ ]:
X1_val.to_numpy().reshape(-1,13,1).shape

(578, 13, 1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution1D, Dropout
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam
from keras.optimizers import SGD

from keras.initializers import random_uniform

#hyperparameters
SEED = 1234
np.random.seed(SEED)

input_dimension = 226
learning_rate = 0.0025
momentum = 0.
hidden_initializer = random_uniform(seed=SEED)
dropout_rate = 0.8

In [ ]:
model = Sequential()
model.add(Convolution1D(32, 3, input_shape= (13, 1), activation='relu'))
model.add(Convolution1D(16, 1, activation='relu'))
model.add(Flatten())
model.add(Dropout(dropout_rate))
model.add(Dense(128, input_dim=input_dimension, kernel_initializer=hidden_initializer, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(64, kernel_initializer=hidden_initializer, activation='relu'))
model.add(Dense(2, kernel_initializer=hidden_initializer, activation='softmax'))

sgd = SGD(lr=learning_rate, momentum=momentum)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, 11, 32)            128       
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 11, 16)            528       
_________________________________________________________________
flatten_7 (Flatten)          (None, 176)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 176)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               22656     
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)               

In [ ]:
m = model.fit(X1_train, Y1_train, epochs=2000, batch_size=30)

Epoch 1/2000
1358/1358 [==============================] - 0s 335us/step - loss: 0.6961 - acc: 0.4713
Epoch 2/2000
1358/1358 [==============================] - 0s 61us/step - loss: 0.6918 - acc: 0.5272
Epoch 3/2000
1358/1358 [==============================] - 0s 60us/step - loss: 0.6887 - acc: 0.5383
Epoch 4/2000
1358/1358 [==============================] - 0s 61us/step - loss: 0.6871 - acc: 0.5405
Epoch 5/2000
1358/1358 [==============================] - 0s 66us/step - loss: 0.6856 - acc: 0.5405
Epoch 6/2000
1358/1358 [==============================] - 0s 59us/step - loss: 0.6879 - acc: 0.5405
Epoch 7/2000
1358/1358 [==============================] - 0s 64us/step - loss: 0.6876 - acc: 0.5405
Epoch 8/2000
1358/1358 [==============================] - 0s 61us/step - loss: 0.6837 - acc: 0.5405
Epoch 9/2000
1358/1358 [==============================] - 0s 61us/step - loss: 0.6855 - acc: 0.5405
Epoch 10/2000
1358/1358 [==============================] - 0s 65us/step - loss: 0.6867 - acc: 0.540

In [ ]:
#testing accuracy
predict_prob_test1 = model.predict(X1_test.reshape(-1,13,1))
test1_scores=np.argmax(predict_prob_test1,axis=1)
(Y1_test==test1_scores).mean()

0.7935010482180294

In [ ]:
#validation accuracy
predict_prob_val1 = model.predict(np.array(X1_val).reshape(-1,13,1))
print(predict_prob_val1)
val1_scores=np.argmax(predict_prob_val1,axis=1)
(Y1_val==val1_scores).mean()

[[0.7494677  0.25053236]
 [0.67890656 0.32109347]
 [0.7086975  0.29130247]
 ...
 [0.08397299 0.916027  ]
 [0.64647907 0.35352084]
 [0.46493113 0.5350688 ]]


0.7422145328719724

In [ ]:
#training accuracy
predict_prob_train1 = model.predict(np.array(X1_train).reshape(-1,13,1))
print(predict_prob_train1)
train1_scores=np.argmax(predict_prob_train1,axis=1)
(Y1_train==train1_scores).mean()

[[0.8065427  0.19345729]
 [0.70209867 0.29790133]
 [0.6367037  0.36329627]
 ...
 [0.73129565 0.2687044 ]
 [0.00478622 0.99521375]
 [0.00644024 0.9935597 ]]


0.7790868924889544

In [ ]:
type(predict_prob_train1)

numpy.ndarray

In [ ]:
predict_prob_train1.shape

(1358, 2)

In [ ]:
Y1_train.shape

(1358,)

In [ ]:
Y1_val.shape

(578,)

In [ ]:
Y1_test.shape

(954,)

In [ ]:
predict_prob_train1.shape

(1358, 2)

In [ ]:
predict_prob_test1.shape

(954, 2)

In [ ]:
predict_prob_val1.shape

(578, 2)

In [ ]:
predict_prob_val1.reshape(578,2).shape

(578, 2)

In [ ]:
for i in predict_prob_val1.flatten():
  print(i.shape)

()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
(

In [ ]:
X1_train.shape

(1358, 13, 1)

In [ ]:
X_train.shape

(2312, 13)

In [ ]:
X1_val.to_numpy().shape

(578, 13)

In [ ]:
X_valid.shape

(578, 13)

Multi-layer Preceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
model = MLPClassifier()

In [ ]:
X2 = data.drop(['Unnamed: 0','Unnamed: 0.1.1','Unnamed: 0.1','final_tweets','label'],axis=1)
y2 = data['label']

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2, test_size=0.33, random_state=14)

In [ ]:
X2= X2.to_numpy()
y2= y2.to_numpy() 
y2= y2.flatten()

In [ ]:
kf = KFold(n_splits=5, random_state=25, shuffle=True)
CV_score_array  =[]
for train_index_2, test_index_2 in kf.split(X):
    X2_train, X2_valid = X2[train_index], X2[test_index]
    y2_train, y2_valid = y2[train_index], y2[test_index]
    mlp=MLPClassifier(activation="relu",alpha=0.00025,hidden_layer_sizes=(200,150,100,50,25),learning_rate='adaptive',solver='adam', max_iter=200, random_state=25)
    mlp.fit(X2_train,y2_train)  

In [ ]:
X2_valid_pred=m.predict(X2_valid)

In [ ]:
X2_valid_prob=m.predict_proba(X2_valid)

In [ ]:
(X2_valid_pred==y2_valid).mean()   #validation accuracy

0.7629757785467128

In [ ]:
X2_test_pred=m.predict(X2_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
X2_test_prob=m.predict_proba(X2_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
X2_test.shape

(954, 13)

In [ ]:
(X2_valid_test==y2_test).mean()    #test accuracy

0.8438155136268344

In [ ]:
X2_train_pred=m.predict(X2_train) 

In [ ]:
X2_train_prob=m.predict_proba(X2_train) 

In [ ]:
(X2_train_pred==y2_train).mean() #training accuracy

0.8213667820069204